In [7]:
import gitProjectExample
# from gitProjectExample.lib import clean_data

In [8]:
from os.path import split
import pandas as pd
import datetime

In [9]:
folder_source, _ = split(gitProjectExample.__file__)

In [13]:
gitProjectExample.__file__

'/Users/arturoguizar/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/gitProjectExample/__init__.py'

In [10]:
folder_source

'/Users/arturoguizar/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/gitProjectExample'

In [12]:
_

'__init__.py'

In [14]:
df = pd.read_csv('{}/data/data.csv.gz'.format(folder_source))

In [15]:
df.head()

,id,civility,birthdate,city,postal_code,vote_1,vote_2,vote_3,vote_4,vote_5,...,qa_2,qa_3,qa_4,qc,qd,qe,qf,surveyduration,Unnamed: 140,Unnamed: 141
0,55330,Mme,23/06/1982 00:00,MERIGNAC,33700,69,60,88,86,75,...,0.0,0.0,0.0,10,3.0,4,5,581,NaN,NaN
1,55334,Mme,04/05/1971 00:00,METZ,57000,66,74,87,91,79,...,0.0,0.0,0.0,5,3.0,4,5,793,NaN,NaN
2,55336,Mll,21/03/1992 00:00,Paris,75007,8,70,86,92,81,...,0.0,0.0,0.0,8,3.0,6,5,831,NaN,NaN
3,55339,Mll,29/10/1985 00:00,hellemmes,59260,28,56,87,91,82,...,NaN,0.0,0.0,10,5.0,2,2,489,NaN,NaN
4,55342,Mll,05/05/1985 00:00,Rosny sous Bois,93110,27,53,85,91,81,...,0.0,0.0,0.0,3,3.0,3,5,845,NaN,NaN


In [20]:
#cols = [x for x in data.columns if x.find('vote') >= 0]
cols = []
for x in df.columns:
    if x.find('vote') >= 0:
        cols.append(x)
cols

['vote_1', 'vote_2', 'vote_3', 'vote_4', 'vote_5', 'vote_6']

In [38]:
cols = [x for x in df.columns if x.find('vote') >= 0]
cols

[]

In [35]:
def clean_data(data):
    """ clean data
    """
    # Remove columns starts with vote
    cols = []
    for x in data.columns:
        if x.find('vote') >= 0:
            cols.append(x)
    
    data.drop(cols, axis=1, inplace=True)
    # Remove special characteres from columns
    data.loc[:, 'civility'] = data['civility'].replace(r'\.', '', regex=True)
    # Calculate Age from day of birth
    actual_year = datetime.datetime.now().year
    data.loc[:, 'Year_Month'] = pd.to_datetime(data.birthdate)
    data.loc[:, 'Age'] = actual_year - data['Year_Month'].dt.year
    # Uppercase variable to avoid duplicates
    data.loc[:, 'city'] = data['city'].str.upper()
    # Take 2 first digits, 2700 -> 02700 so first two are region
    data.loc[:, 'postal_code'] = data.postal_code.str.zfill(5).str[0:2]
    # Remove columns with more than 50% of nans
    cnans = data.shape[0] / 2
    data = data.dropna(thresh=cnans, axis=1)
    # Remove rows with more than 50% of nans
    rnans = data.shape[1] / 2
    data = data.dropna(thresh=rnans, axis=0)
    # Discretize based on quantiles
    data.loc[:, 'duration'] = pd.qcut(data['surveyduration'], 10)
    # Discretize based on values
    data.loc[:, 'Age'] = pd.cut(data['Age'], 10)
    # Rename columns
    data.rename(columns={'q1': 'Frequency'}, inplace=True)
    # Transform type of columns
    data.loc[:, 'Frequency'] = data['Frequency'].astype(int)
    # Rename values in rows
    drows = {1: 'Manytimes', 2: 'Onetimebyday', 3: '5/6timesforweek',
             4: '4timesforweek', 5: '1/3timesforweek', 6: '1timeformonth',
             7: '1/trimestre', 8: 'Less', 9: 'Never'}
    data.loc[:, 'Frequency'] = data['Frequency'].map(drows)
    return data

In [36]:
clean_data = clean_data(df)

In [37]:
clean_data.head()

,id,civility,birthdate,city,postal_code,Frequency,q10_1,q10_2,q10_3,q10_4,...,qa_3,qa_4,qc,qd,qe,qf,surveyduration,Year_Month,Age,duration
0,55330,Mme,23/06/1982 00:00,MERIGNAC,00,1/3timesforweek,2.0,2.0,2.0,1.0,...,0.0,0.0,10,3.0,4,5,581,1982-06-23,"(34.4, 40.1]","(519.8, 582.0]"
1,55334,Mme,04/05/1971 00:00,METZ,00,Manytimes,1.0,1.0,2.0,1.0,...,0.0,0.0,5,3.0,4,5,793,1971-04-05,"(45.8, 51.5]","(778.4, 852.0]"
2,55336,Mll,21/03/1992 00:00,PARIS,00,Manytimes,2.0,1.0,1.0,3.0,...,0.0,0.0,8,3.0,6,5,831,1992-03-21,"(22.943, 28.7]","(778.4, 852.0]"
3,55339,Mll,29/10/1985 00:00,HELLEMMES,00,Onetimebyday,NaN,NaN,NaN,NaN,...,0.0,0.0,10,5.0,2,2,489,1985-10-29,"(34.4, 40.1]","(438.8, 519.8]"
4,55342,Mll,05/05/1985 00:00,ROSNY SOUS BOIS,00,4timesforweek,2.0,2.0,1.0,2.0,...,0.0,0.0,3,3.0,3,5,845,1985-05-05,"(34.4, 40.1]","(778.4, 852.0]"


In [24]:
df.shape

(999, 138)

In [27]:
assert df.shape == (985, 119)

AssertionError: 

In [28]:
assert df.shape == (999, 138)

In [32]:
(df['civility'].replace(r'\.', '', regex=True)).head()

0    Mme
1    Mme
2    Mll
3    Mll
4    Mll
Name: civility, dtype: object